# TODO
- vP is not allow to be 0 or 1
- DONE add everything to dictionaries
- DONE attacker only works with dict
- optimise
- Max it is dependent on deltaB
- DONE t must not be prime --> optimise
- DONE try catch when non invertible prime
- DONE when there is no R large/ small en
- DONE when delta of bounds is small then the possibility of a new generation of p is high 
- DONE when p is smaller than there is is higher chance of success
- Problem when m>=n 
- Use secret module instead of the random module for cryptographic purposes
- DONE camelCase to snake_case if required
- gen_prime() is not used
- Check if N/P is not a hole number

### Import and Configuration

In [26]:
import sympy
import random
import coloredlogs, logging

In [27]:
coloredlogs.install(fmt='%(levelname)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [28]:
MIN_PRIME = 0
MAX_PRIME = 1024
OPT_RANGE_REDUCTION = 10

MAX_ITERS = 10

msg = 12345678900987654321

In [29]:
def gen_prime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def gen_prime_with_optimized_bit_limit():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def gen_prime_with_boundaries(lb, ub):
    return sympy.randprime(lb, ub)

def gen_prime_half_size():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def gen_random_number_with_bit_limit():
    return random.randint(0, pow(2, MAX_PRIME))

def fermat_prime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def modular_multiplicative_inverse(E, phiN):
    return pow(E, -1, phiN)

def calc_n(P, Q):
    return P*Q

def calc_phi_n(P, Q):
    return (P-1) * (Q-1)

def to_binary(dec):
    return "{0:b}".format(dec)

def to_decimal(bin):
    return int(bin, 2)

def concatenate_in_binary(prev, tail):
    prev = to_binary(prev)
    tail = to_binary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return to_decimal(prev + tail)

def split_in_binary(bin):
    bin = to_binary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(to_decimal(prev), to_decimal(tail))


# Voraussetzungen

In [30]:
def create_attacker_key():
    a_P = gen_prime_half_size()
    a_Q= gen_prime_half_size()


    a_PhiN = calc_phi_n(a_P, a_Q)
    a_N = calc_n(a_P, a_Q)

    a_E = fermat_prime()
    a_D = modular_multiplicative_inverse(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

attacker = {}

created = False
while not created:
    try:
        attacker = create_attacker_key()
        created = True
    except ValueError:
            logger.error("Trying to generate Key failed. Trying again ...")
logger.info("Attacker Key successfully generated!")

ERROR Trying to generate Key failed. Trying again ...
INFO Attacker Key successfully generated!


# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [31]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
user = {}

valid = False
while not valid:
    # Schritt 1
    P = gen_prime_with_optimized_bit_limit()
    vP = pow(P, attacker["E"] ,attacker["N"])

    # Schritt 2
    t = gen_random_number_with_bit_limit() 
    tempN = concatenate_in_binary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    ub = int(int(tempN)//int(P)) 
    lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    logger.info("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    R = 0
    Q = 0
    try:
        for i in range(MAX_ITERS):
            Q = int(gen_prime_with_boundaries(lb, ub))
            R = tempN - P*Q

            if (R < pow(2,MAX_PRIME+1) and R >= 0):
                valid = True
                break
            r_too_large += 1
    except ValueError:
        pass
    user["P"] = P
    user["Q"] = Q

attacker["vP"] = vP
attacker["R"] = R

logger.info("Amount of failed attempts: {0}".format(r_too_large))

logger.info("tempN = P * Q + R")
logger.info("{0} = {1} * {2} + {3}".format(tempN, P, Q, R))

INFO Bounds: [104628508878367140489121283185997026671845897010392893577780257813426245859012354027393435094580421056109878532579025489411511302802294375580328384331560248310946337854156337442357401966742782532646799867406255963137928143108571126106484685786890892950451389159873115636548078836645547238369688640425199356207266, 104628508878367140489121283185997026671845897010392893577780257813426245859012354027393435094580421056109878532579025489411511302802294375580328384331560248310946337854156337442357401966742782532646799867406255963137928143108571126106484685786890892950451389159873115636548078836645547238369688640425199356211342] 	 P: 44101604698226363752744081825648087219208423112788645662266832456875120218136578139196798229919413050332615161719098447938536802454871176102606869168667088796043372146312407349382789005815777646388152247710547501366387901732127527075052950074356919014746727310524146700941628247556676831913094887272153083 	 deltaB: 4076
INFO Amount of failed attempts: 

## Schritt 4

In [32]:
user["N"] = calc_n(user["P"], user["Q"])

## Schritt 5

In [33]:
def create_user_key():
    PhiN = calc_phi_n(P, Q)
    E = fermat_prime()
    D = modular_multiplicative_inverse(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        user.update(create_user_key())
        created = True
    except ValueError:
            logger.info("Trying to generate Key failed. Trying again ...")
logger.info("User Key successfully generated!")

INFO Trying to generate Key failed. Trying again ...
INFO Trying to generate Key failed. Trying again ...
INFO Trying to generate Key failed. Trying again ...
INFO Trying to generate Key failed. Trying again ...
INFO User Key successfully generated!


## Schritt 6

In [34]:
cipher = pow(msg, user["E"], user["N"])
signature = pow(msg, user["D"], user["N"])

# Angriff

In [35]:
class Calc_Secret_Result():
    was_successfull = True
    key_parameter = {}

In [36]:
def calc_secrets(vP):
    calc_secret_result = Calc_Secret_Result()

    P = pow(vP, attacker["D"], attacker["N"])
    # Potential to optimize when N/P not hole Number
    Q =  int(pub_user["N"]//P)
    PhiN =  int(calc_phi_n(P, Q))
    try:
        D = modular_multiplicative_inverse(pub_user["E"], PhiN)
        calc_secret_result.key_parameter = {
        "P" : P,
        "Q" : Q,
        "PhiN" : PhiN,
        "D" : D, 
    }
    except ValueError:
        calc_secret_result.was_successfull = False
        logger.info("Trying to generate Key failed. Trying again ...")
    
    return calc_secret_result

In [37]:
def validate_through_signature(key_parameter):
    key_parameter["Sig"] = pow(msg, key_parameter["D"], collection_user["N"])
    if (key_parameter["Sig"] == signature):
        collection_user.update(key_parameter)
        return True
    return False
    

## Schritt 1

In [38]:
pub_user = {
    "N" : user["N"],
    "E" : user["E"]
}

collection_user = {
    "N" : pub_user["N"],
    "E" : pub_user["E"]
}

## Schritt 2

In [39]:
collection_user["vP"] = split_in_binary(user["N"])[0]
collection_user["vP'"] = collection_user["vP"]+1

In [40]:
secrets_for_vP = calc_secrets(collection_user["vP"])
secrets_for_vP_plus_one = calc_secrets(collection_user["vP'"])

if(secrets_for_vP.was_successfull and secrets_for_vP_plus_one.was_successfull):
    logger.info("Both key parameters of D were successfully calculated.")
    if(validate_through_signature(secrets_for_vP.key_parameter)):
        logger.info("D is the right privat Key: {0}".format(collection_user["D"]))
    elif(validate_through_signature(secrets_for_vP_plus_one.key_parameter)):
        logger.info("D' is the right privat Key: {0}".format(collection_user["D"]))
elif(not secrets_for_vP.was_successfull and not secrets_for_vP_plus_one.was_successfull):
    logger.error("Both key parameters calculated with errors!")
elif(not secrets_for_vP_plus_one.was_successfull):
    logger.info("Second key parameters calculated with error! First key parameters must be correct!")
    if(validate_through_signature(secrets_for_vP.key_parameter)):
        logger.info("D is the right privat Key: {0}".format(collection_user["D"]))
elif(not secrets_for_vP.was_successfull):
    logger.info("First key parameters calculated with error! Second key parameters must be correct!")
    if(validate_through_signature(secrets_for_vP_plus_one.key_parameter)):
        logger.info("D' is the right privat Key: {0}".format(collection_user["D"]))
else:
    logger.error("Logical programming error!")


INFO Both key parameters of D were successfully calculated.
INFO D is the right privat Key: 3985829317606840703268098741734909618975264319208459923866030246007814421418873296281927780162166250630546631847318760419337345920499480584986624149886996505849012433746841086393377782716900527488561476577858674153199542379057852711751651976195600620570111348684888144504554867126976552836927771515388045356752021286560788153237525225140464322161457638232950402754281982254379778959672749239686993929369807491208230681997099651366675324870534620675342629767725787481370524290492906380533392800405447582830900046237067068625407207843612553583350134655351455096492955233662296844970667893876655419822088815153153989421


# Result

In [41]:
def sort_dic(dict):
    return {key: dict[key] for key in sorted(dict)}

In [42]:
# logger.info(sort_dic(attacker))
# logger.info(sort_dic(user))
# logger.info(sort_dic(collection_user))

# Validation Tests

### Validation test for vP

In [43]:
testVar = "vP"
altTestVar = "vP'"

if(collection_user[testVar] == attacker[testVar] or collection_user[altTestVar] == attacker[testVar]):
    logger.info("Success!!!")
    logger.info("The variable {0} or {1} is in the expected state: {2}".format(testVar, altTestVar, attacker[testVar]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(attacker[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variable vP or vP' is in the expected state: 25667813094652664678105323386650046670211406592660962528437406172636076011248471485001769944541616199674190471574125877658757228825525038339219578297298337148394877067643262530241967941063138844259374529552129558875144289285805926925204330258124047915025106848187160633054620199920295006390262685982430646847
INFO Current vP: 25667813094652664678105323386650046670211406592660962528437406172636076011248471485001769944541616199674190471574125877658757228825525038339219578297298337148394877067643262530241967941063138844259374529552129558875144289285805926925204330258124047915025106848187160633054620199920295006390262685982430646847 	 Current vP': 2566781309465266467810532338665004667021140659266096252843740617263607601124847148500176994454161619967419047157412587765875722882552503833921957829729833714839487706764326253024196794106313884425937452955212955887514428928580592692520433025812404791502510684818716063305462019992

### Validation test for P and P'

In [44]:
testVar = "P"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state:".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variables P are in the expected state:
INFO Current P: 44101604698226363752744081825648087219208423112788645662266832456875120218136578139196798229919413050332615161719098447938536802454871176102606869168667088796043372146312407349382789005815777646388152247710547501366387901732127527075052950074356919014746727310524146700941628247556676831913094887272153083


### Validation test for Q and Q'

In [45]:
testVar = "Q"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state!".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], attacker[testVar]))

INFO Success!!!
INFO The variables Q are in the expected state!
INFO Current Q: 104628508878367140489121283185997026671845897010392893577780257813426245859012354027393435094580421056109878532579025489411511302802294375580328384331560248310946337854156337442357401966742782532646799867406255963137928143108571126106484685786890892950451389159873115636548078836645547238369688640425199356210139


### Validation test for PhiN and PhiN'

In [46]:
testVar = "PhiN"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state:".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variables PhiN are in the expected state:
INFO Current PhiN: 4614285138718615095477581869903036012414228642630669623048665881765277662230462263843223064925330581822863659207181106173749123661333918480476733972393070074788057583693393938968801111902589777075442113555371286993309399390512700591229054698979545987004352289462498707431329818001813469878181543548144112072661801046745939361585693428486170713845285355087754509641366135044519396825353252317064996575738091069779968808288917698885690073767293060274503717070665505540735546986028044116179134160501878049071535502468401273189014717642275104205755379253285904023226207047190933711272414579516248895954500621410896345316


### Validation test for D and D'

In [47]:
testVar = "D"
altTestVar = "D'"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state:".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variables D are in the expected state:
INFO Current D: 3985829317606840703268098741734909618975264319208459923866030246007814421418873296281927780162166250630546631847318760419337345920499480584986624149886996505849012433746841086393377782716900527488561476577858674153199542379057852711751651976195600620570111348684888144504554867126976552836927771515388045356752021286560788153237525225140464322161457638232950402754281982254379778959672749239686993929369807491208230681997099651366675324870534620675342629767725787481370524290492906380533392800405447582830900046237067068625407207843612553583350134655351455096492955233662296844970667893876655419822088815153153989421


### Validation test for signature and signature'

In [48]:
testVar = "Sig"

if(collection_user[testVar] == signature):
    collection_user["correct_key"] = "D"
    logger.info("D is the right privat Key: {0}".format(collection_user[testVar]))
else:
    logger.error("Message was incorrectly signed!")

INFO D is the right privat Key: 917754571566529405814565155805080837187650793096425767181901436161226074358904434589407238394392725276941835304692540942427039993012432449229683077013873560198250530620523566131330653469789217165571528414270924594016737603245184979406521189197283627392430314483720095609998043873322279065702819700958291563189152314995934633988193472903311591572363633100358380334101677683943311360448639431046056226631057392903931576495324250239163145219424877204783444601135971998243913852007651664286560734617549098485931788013992027475716300419267697646647053095736887338165498622898577250827356296844150952704832746067561617723
